In [15]:
from datetime import datetime


In [16]:
def process(log_name):
    print(f'---{log_name}---')
    logs = {}
    earliest_task_start_time = None
    latest_submit_time = None
    latest_finish_time = None
    with open(log_name, 'r') as f:
        lines = f.readlines()
        count = 0
        for line in lines:
            if "job task started" in line:
                splits = line.split(' ')
                ts_string = splits[1] + ' ' + splits[2]
                dt_object = datetime.strptime(ts_string, '%Y-%m-%d %H:%M:%S,%f')
                uid = splits[6]
                node = splits[-1].rstrip('\n')
                logs[uid] = {'started': (node, dt_object)}
                if earliest_task_start_time is None or dt_object < earliest_task_start_time:
                    earliest_task_start_time = dt_object
        for line in lines:
            if "is submitted" in line:
                splits = line.split(' ')
                ts_string = splits[1] + ' ' + splits[2]
                dt_object = datetime.strptime(ts_string, '%Y-%m-%d %H:%M:%S,%f')
                uid = splits[5]
                logs[uid]['submitted'] = (dt_object)
                if latest_submit_time is None or dt_object > latest_submit_time:
                    latest_submit_time = dt_object
        for line in lines:
            if "finished" in line:
                splits = line.split(' ')
                ts_string = splits[1] + ' ' + splits[2]
                dt_object = datetime.strptime(ts_string, '%Y-%m-%d %H:%M:%S,%f')
                uid = splits[6]
                node = splits[-1].rstrip('\n')
                logs[uid]['finished'] = (node, dt_object)
                if latest_finish_time is None or dt_object > latest_finish_time:
                    latest_finish_time = dt_object
        # the start time is defined to be the earlier of {last submitted time, first start task time}
        start_time = latest_submit_time
        if earliest_task_start_time < start_time:
            start_time = earliest_task_start_time
        # total time is defined to be the difference between the start time and the latest task finish time
        total_time = (latest_finish_time - start_time).total_seconds()
        print('total time: ', total_time)

        from statistics import mean
        count = {}
        execute_times = []
        wait_times = []

        for uid in logs:
            log = logs[uid]
            # check if tasks are distributed evenly
            if log['started'][0] in count.keys():
                count[log['started'][0]] += 1
            else:
                count[log['started'][0]] = 1
            # execute time = task finished - task started
            execute_time = (log['finished'][1] - log['started'][1]).total_seconds()
            execute_times.append(execute_time)
        print('task distribution: ', count)
        print('avg execute time: ', mean(execute_times))

In [17]:
for i in range(5, 11):
    process(f'debug.log.{i}')

---debug.log.5---
total time:  92.475
task distribution:  {'celery@xgpg0': 34, 'celery@xgpg1': 36, 'celery@xgpg2': 30}
avg execute time:  18.21617
---debug.log.6---
total time:  128.261
task distribution:  {'celery@xgpg0': 50, 'celery@xgpg1': 50}
avg execute time:  17.83018
---debug.log.7---
total time:  235.426
task distribution:  {'celery@xgpg0': 100}
avg execute time:  17.58116
---debug.log.8---
total time:  422.744
task distribution:  {'celery@xgpg0': 100}
avg execute time:  16.32399
---debug.log.9---
total time:  859.237
task distribution:  {'celery@xgpg0': 100}
avg execute time:  16.71336
---debug.log.10---
total time:  1558.811
task distribution:  {'celery@xgpg0': 100}
avg execute time:  15.16555


In [18]:
# Scalability test
# 1 nodes: 235.426s (baseline)
# 2 nodes: 128.261s (91.78%)
# 3 nodes: 92.475s (84.86%)

# Load balancing test
# 2 nodes: {'celery@xgpg0': 50, 'celery@xgpg1': 50}
# 3 nodes: {'celery@xgpg0': 34, 'celery@xgpg1': 36, 'celery@xgpg2': 30}

# Per-worker concurrency test
# 1: 1558.811s (baseline)
# 2: 859.237s (90.71%)
# 4: 422.744s (92.18%)
# 8: 235.426s (82.77%)